**Warning**: This scraping script is useless now, since Sing Tao Daily changed who they archive their articles.

**Purpose**: Scraping for Sing Tao Daily articles from June 1, 2019 to Dec 30, 2019. 

**Goal**: A set of csv files with all Sing Tao Daily articles between the specified date range.

In [1]:
import bs4 as BeautifulSoup
from urllib.request import Request, urlopen
import re
import pandas as pd
import time
import datetime

In [8]:
sing_tao_archive_htm = "C:/Users/sunny/Desktop/news_nlp/std_news.htm"   # This htm file is a search result page generated 
                                                                        # by the Sing Tao Daily archive tool.
                                                                        # It contains a list of articles links.
sing_tao_daily_folder = 'C:/Users/sunny/Desktop/news_nlp/sing_tao_daily_collection/'

In [3]:
# formatted current time

def current_datetime():
    return datetime.datetime.now().strftime("%Y/%m/%d - %H:%M:%S")

In [4]:
# read the html page to soup

def page2soup(url):
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    page = urlopen(req).read()
    soup = BeautifulSoup.BeautifulSoup(page, "html.parser")
    return soup

In [17]:
# extract content from the article url

def extract_content(url):
    content_text=''
    try:
        soup = page2soup(url)
        title = soup.find('div', class_="post-heading supplement-p-h").find('h1').text.strip()
        print('\t\t'+current_datetime() + ' : '+title)
        author = soup.find('meta', attrs={'name': 'author'})["content"]
        date_cat = soup.find('div', class_="post-heading supplement-p-h").find('div',class_="date").text.strip()
        date = date_cat[:10]
        category = date_cat[10:]
        for content in soup.find_all("div", class_="post-content"):
            for p in content.find_all('p'):
                content_text = content_text + p.text
        one_article = [title,author,category,date,url,content_text]
        return(one_article)
    except:
        print('Error777: Can not extract content : '+ str(url))
        return(['no_title','no_author','no_category','1000-01-01',url,'no_content'])

In [18]:
# save articles to csv files

def export_to_csv(news_collection, c):
    df = pd.DataFrame(news_collection, columns=['Title', 'Author', 'Category', 'Date', 'Link', 'Content'])
    filename = sing_tao_daily_folder + 'sing_tao_daily_batch' + c + '.csv'
    export_csv = df.to_csv(filename, index=None, header=True)
    print(current_datetime() + ' : Exported batch ' + c + ' to csv.')

In [16]:
'''
Purpose: Load the pre-downloaded htm file and parse it
Source: This file is manually downloaded from 星島日報 website
using the site's dynamic search function.

Filter date range : Jun 1, 2019 to Dec 31, 2019
Filter category: 
hk 港聞 True
wor 國際 True
spt 體育
fin 財經 True
chi 中國 True
ent 娛樂
rac 馬經
edu 教育 True
pp 地產 True
edi 社論 True
ls 副刊
'''
with open("C:/Users/sunny/Desktop/news_nlp/std_news.htm","r", encoding="utf8") as f:
    contents = f.read()
    soup = BeautifulSoup.BeautifulSoup(contents, "html.parser")

search_result = soup.find("div",class_="tab-content",id = "search-result")
result_by_category = search_result.find_all("div",role="tabpanel")

# Extract all articles' link.
article_ls=[]
for c in result_by_category:
    category= c.get("id")
    articles = c.find_all("a")
    for article in articles:
        title = article.text.strip()
        link = article["href"]
        one_article = [category, title, link]
        article_ls.append(one_article)

print('Total articles: ' + str(len(article_ls)))

Total articles: 15130


In [20]:
# Construct a news collection.

def archive_news_collection(i):
    print(current_datetime() + ' : Start extracting Sing Tao Daily news.')
    news_collection = []
    c=1
    article_ls_len = len(article_ls[i:])

    for a in article_ls[i:]:
        try:
            url = a[2]
            article = extract_content(url)
            news_collection.append(article)

            # export news collection every hundred articles to save progress
            if (c % 100 ==0 or c == article_ls_len):
                export_to_csv(news_collection, c)
                news_collection = []
            c+=1
        except:
            print(current_datetime()+' : run into error.')
            c+=1
            time.sleep(3)
            continue
    print(current_datetime() + ' : Finished extracting news.')

In [ ]:
archive_news_collection(0) # in case the loop fail, we can restart the download at a specific place in the list.